# 패키지 임포트

In [8]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Attention
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from transformer_build import  *
from data_preprocessing import *
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

# 데이터 로드

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [ ]:
textcuboid=np.load('./1-Channel textcuboid_20ng(elmo).npy')
textcuboid_test=np.load('./1-Channel textcuboid_test_20ng(elmo).npy')

# 전처리

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [4]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [5]:
X = df_newsgroup['text'].astype(str)
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [6]:
x_train=list(X_train)
x_test=list(X_test)

sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)+1 #1을 더해야 에러가 안터짐 토큰 영향으로 보임

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 300

xtext_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
xtext_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

In [9]:
one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

In [17]:
textcuboid_train=textcuboid[1000:]
text_train=xtext_train[1000:]
textcuboid__val=textcuboid[:1000]
text_val=xtext_train[:1000]
y_train=one_hot_train_labels[1000:]
y_val=one_hot_train_labels[:1000]

text_test=xtext_test
y_test=one_hot_test_labels

# 모델  빌드

In [13]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [10]:
embedding_dim = 384 # 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
num_transformer_blocks = 1

#ELMo channel의 학습
ELMo_input = Input(shape=(952, 256))
conv1 = Conv1D(256, 1, padding='valid', activation='relu')(ELMo_input)
pooling = GlobalMaxPooling1D()(conv1)
# pooling = Dropout(0.5)(pooling)
ELMo_x = Dense(256, activation='relu')(pooling)
# ELMo_attention_layer = Attention()
# ELMo_a=ELMo_attention_layer([ELMo_x,ELMo_x])

#Sequence channel의 학습
Text_inputs = Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
Text_x = embedding_layer(Text_inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    Text_x = transformer_block(Text_x)

Text_x = tf.keras.layers.GlobalAveragePooling1D()(Text_x)
Text_x = tf.keras.layers.Dropout(0.5)(Text_x)
Text_x = tf.keras.layers.Dense(256, activation="relu")(Text_x)
# Text_attention_layer = Attention()
# Text_a=Text_attention_layer([Text_x,Text_x])

#결합 후 분류
# x = layers.concatenate([ELMo_x,ELMo_a, Text_x,Text_a], axis=-1)
# x = layers.concatenate([ELMo_a, Text_a], axis=-1)
x = layers.concatenate([ELMo_x, Text_x], axis=-1)
# x = Dropout(0.5)(x)
output_layer = Dense(20, activation='softmax')(x)

model = Model(inputs=[ELMo_input, Text_inputs], outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 token_and_position_embedding (  (None, 300, 384)    58118400    ['input_2[0][0]']                
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 input_1 (InputLayer)           [(None, 952, 256)]   0           []                               
                                                                                                  
 transformer_block (Transformer  (None, 300, 384)    617888      ['token_and_position_embeddin

In [18]:
history = model.fit([textcuboid_train,text_train], y_train,callbacks=callbacks, epochs=50,batch_size=256,validation_data=([textcuboid__val,text_val], y_val))

Epoch 1/50
55/55 [==============================] - 16s 240ms/step - loss: 2.2173 - accuracy: 0.3561 - val_loss: 1.0749 - val_accuracy: 0.6940 - lr: 0.0010
Epoch 2/50


ResourceExhaustedError: Graph execution error:

Detected at node 'model/transformer_block_3/dropout_7/dropout/Mul' defined at (most recent call last):
    File "C:\ProgramData\anaconda3\envs\class\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\anaconda3\envs\class\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\anaconda3\envs\class\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\anaconda3\envs\class\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\ProgramData\anaconda3\envs\class\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "C:\ProgramData\anaconda3\envs\class\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramD

In [ ]:
model.evaluate([textcuboid_test,text_test],y_test)

In [ ]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model_45.h5', custom_objects=custom_objects)
model.evaluate([textcuboid_test,text_test],y_test)